In [7]:
library(data.table)
library(dplyr)
library(stringr)
library(plyr)
library("ggpubr")

#write log regression result

##use this if you dont have train.txt

#result_outrider <- fread("pred_no_na.txt")

#colnames(result_outrider)[1] <- "featureID"
#dim(result_outrider)


#measured <- read.table("prot_int.csv")

#measured[measured != 0] <- 1
#measured$featureID <- rownames(measured)


#prot_int_list <- melt(result_outrider)
#colnames(prot_int_list)[2:3] <- c("sampleID","protein_intensity")


#measured_list <- melt(measured)
#colnames(measured_list)[2:3] <- c("sampleID","measured")


#train <- merge(prot_int_list, measured_list)
#log_reg <- glm(measured~protein_intensity,family=binomial,data=train)
#train$predprob<-round(fitted(log_reg),2)
#train$missing <- 1- train$predprob

#write.table(train, "train.txt", sep = "\t", col.names = T,row.names = F, quote = F)



#read result from var_maf and train
var_maf <- read.table("variant_with_maf_new.csv",sep =",", header = T)
outliers <- read.table("train.txt",header = T)

print("reading done!")

[1] "reading done!"


In [8]:
#get cellline name

cellline_names <- var_maf$sampleID %>%
  strsplit( ":")  %>%
  sapply( tail, 1 ) %>%
  str_replace_all("-","")%>%
  str_replace_all(" ","")%>%
  str_replace("(TB)","")%>%
  str_replace("UO31","U031")%>%
  str_replace_all("\\(|\\)", "")%>%
  str_replace_all("A549/ATCC","A549")%>%
  str_replace_all("7860","X786O")%>%
  str_replace_all("MDAN","MDAMB468")%>%
  str_replace_all("NCI/ADRRES","NCIADRES")

var_maf$cellline_names <- cellline_names


#get celline name from outlier result
cellline_names_outliers <- sapply(strsplit(outliers$sampleID ,"_"), head, 1)

outliers$cellline_names <-cellline_names_outliers


In [9]:
#head(var_maf)
#check number of celllines for var_maf #60
length(unique(var_maf$cellline_names))

#check number of celllines for outliers #60
length(unique(outliers$cellline_names))

#check if they are identical

SameElements <- function(a, b) return(identical(sort(a), sort(b)))

SameElements(unique(var_maf$cellline_names),unique(outliers$cellline_names))

[1] 60

[1] 60

[1] TRUE

In [10]:
### get gene name

#load necessary packages
library(biomaRt)
library(dplyr)


#extract most possible protein id
outliers$gene_id <-  sapply(strsplit(outliers$featureID ,";"), head, 1)

outliers$gene_id <- gsub(".*\\__", "", outliers$gene_id) 

outliers$gene_id <- gsub("\\-.*", "", outliers$gene_id)

#define list of Uniprot IDs
uniprot_ids <- unique(outliers$gene_id[!(startsWith(outliers$gene_id, "ENSEMBL")|startsWith(outliers$gene_id, "REFSEQ"))])
length(unique(uniprot_ids))

#define list of RefSeq IDs
refseq_ids <- sapply(strsplit(outliers$gene_id[startsWith(outliers$gene_id, "REFSEQ")] ,":"), tail, 1) %>% unique()

#define list of ensembl IDs
ensembl_ids <- sapply(strsplit(outliers$gene_id[startsWith(outliers$gene_id, "ENSEMBL")] ,":"), tail, 1) %>% unique()
head(uniprot_ids)

[1] 6803

[1] "A0AV96" "A0AVF1" "A0AVT1" "A0FGR8" "A0JLT2" "A0JNW5"

In [11]:
length(unique(uniprot_ids))
length(unique(refseq_ids))
length(unique(ensembl_ids))

[1] 6803

[1] 2

[1] 10

In [12]:
# only uniprot_ids used. 99.8% of found ids
length(uniprot_ids)/(length(uniprot_ids)+ length(refseq_ids)+length(ensembl_ids))

[1] 0.9982392

In [13]:
nrow(var_maf)

[1] 352827

In [14]:
##Search in DB gene chromosom position

#use biomaRt package
mart <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")

#convert Uniprot IDs to chromosome regions

uniport_results <- getBM(attributes = c("chromosome_name","start_position","end_position","uniprotswissprot"), filters = "uniprotswissprot", values = uniprot_ids, mart = mart)

#store results in a dataframe
uniport_results_df <- data.frame(uniport_results)
#nrow(uniport_results_df)
#nrow(uniport_results_df$uniprotswissprot[unique(uniport_results_df$uniprotswissprot)])

#remove chr not a number
uniport_results_df_cl <- uniport_results_df[uniport_results_df$chromosome_name %in% c(c(1:22) %>% as.character(),"X","Y"),]

nrow(uniport_results_df_cl)/nrow(uniport_results_df) #8% weg


#display dataframe
#print(uniport_results)



#convert refseq IDs to chromosome regions
refseq_results <- getBM(attributes = c("chromosome_name","start_position","end_position","refseq_mrna"), filters = "refseq_mrna", values = refseq_ids, mart = mart)

#store results in a dataframe
refseq_results_df <- data.frame(refseq_results)

#display dataframe (empty)
#print(refseq_results_df)



#convert ensembl IDs to chromosome regions
ensembl_results <- getBM(attributes = c("chromosome_name","start_position","end_position","ensembl_transcript_id"), filters = "ensembl_gene_id", values = ensembl_ids, mart = mart)

#store results in a dataframe
ensembl_results_df <- data.frame(ensembl_results)

#display dataframe (empty)
#print(ensembl_results_df)

#### uniport outliers

uniport_outliers <- outliers[!(startsWith(outliers$gene_id, "ENSEMBL")|startsWith(outliers$gene_id, "REFSEQ")),]

colnames(uniport_outliers)[8] <- "uniprotswissprot"
uniport_outliers_result <-join(uniport_outliers, uniport_results_df_cl, type="full",by = "uniprotswissprot") %>% na.omit()



[1] 0.92088

In [15]:
nrow(uniport_outliers_result)/max(nrow(uniport_results_df_cl),nrow(uniport_outliers)) #1% filtered out

[1] 0.9908086

In [16]:
head(uniport_outliers)

,featureID,sampleID,protein_intensity,measured,predprob,missing,cellline_names,uniprotswissprot
,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>
1,A0AV96;A0AV96-2,X786O_NCI60,26.95893,1,0.97,0.03,X786O,A0AV96
2,A0AV96;A0AV96-2,A498_NCI60,28.67702,1,0.99,0.01,A498,A0AV96
3,A0AV96;A0AV96-2,A549_NCI60,29.12149,1,0.99,0.01,A549,A0AV96
4,A0AV96;A0AV96-2,ACHN_NCI60,29.22591,1,0.99,0.01,ACHN,A0AV96
5,A0AV96;A0AV96-2,BT549_NCI60,24.08905,1,0.86,0.14,BT549,A0AV96
6,A0AV96;A0AV96-2,CAKI1_NCI60,27.47051,1,0.98,0.02,CAKI1,A0AV96


In [17]:

#extract chromosom number and position from var_maf

var_maf$chromosome_name <- sapply(strsplit(var_maf$Identifier,":"), head, 1) %>% str_replace("chr","")

var_maf$position <- gsub(".*:(.+)_._.*", "\\1", var_maf$Identifier)

In [18]:
#map chromosomes

match_gene_id <- function(A, B) {
  # Add new column to B dataframe to store the matched gene ID
  B$uniprotswissprot <- NA
  #B$uniprotswissprot_2 <- NA
  
  # Loop through each row in B
  for (i in 1:nrow(B)) {
    # Subset A to only include rows with the same chromosome name as B
    A_subset <- subset(A, A$chromosome_name == B$chromosome_name[i])
    #Check if the position of SNP is between start and end position
    A_subset <- A_subset[A_subset$start_position <= B$position[i] & A_subset$end_position >= B$position[i],]
    #if there is a match assign the gene_id to the B dataframe
    if(nrow(A_subset) > 0){
      #print(A_subset$uniprotswissprot)
      B$uniprotswissprot[i] <- A_subset$uniprotswissprot[1]
      #if(length(A_subset$uniprotswissprot) > 1){
      #    B$uniprotswissprot_2[i] <- A_subset$uniprotswissprot[2]
      #  }
      
    }else{
      B$uniprotswissprot[i] <- "None_found"
    }
    
  }
  return(B)
}

#mapping start
#test <-  match_gene_id(uniport_results_df_cl,var_maf)

var_maf_with_gene <- match_gene_id(uniport_results_df_cl,var_maf)

var_maf_with_gene_cl <- var_maf_with_gene[var_maf_with_gene$uniprotswissprot != "None_found",]

var_maf_outlier_result <-join(uniport_outliers, var_maf_with_gene_cl,by = c("uniprotswissprot","cellline_names"))

var_maf_outlier_result_cl <- na.omit(var_maf_outlier_result)


#write.table(var_maf_outlier_result_cl, "outlier_prob_with_maf.txt",quote = F,row.names = F,col.names = T,sep = ",")


In [23]:
write.table(var_maf_outlier_result_cl, "outlier_prob_with_maf_v1.txt",quote = F,row.names = F,col.names = T,sep = ",")

In [ ]:
get

In [22]:
nrow(var_maf_outlier_result_cl)/max(nrow(uniport_outliers),nrow(var_maf_with_gene_cl))

[1] 0.4227823